# CSGO Utility Analysis

## Initialisation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import matplotlib.colors as colors
import os
warnings.filterwarnings('ignore')
%matplotlib inline

MAP_IMAGES_PATH = '../csgo_mm_data/map_images'
MAP_COORD_DATA_PATH = '../csgo_mm_data/map_data.csv'

ESEA_DEMO_PATH = '../csgo_mm_data/esea_demos'
MM_DEMO_PATH = '../csgo_mm_data/mm_demos'

WRITE_FILES = False

COMPETITIVE_MAPS = ['mirage', 'inferno', 'overpass', 'cache','nuke','train','dust2','cbble']
COMPETITIVE_MAPS = [f"de_{map_name}" for map_name in COMPETITIVE_MAPS]


#Convert map coordinates to image coordinates, from Bill Freeman's analysis
def pointx_to_resolutionx(xinput,startX=-3217,endX=1912,resX=1024):
    sizeX = endX - startX
    if startX < 0:
        xinput += startX * (-1.0)
    else:
        xinput += startX
    xoutput = float((xinput / abs(sizeX)) * resX);
    return xoutput


def pointy_to_resolutiony(yinput,startY=-3401,endY=1682,resY=1024):
    sizeY=endY-startY
    if startY < 0:
        yinput += startY *(-1.0)
    else:
        yinput += startY
    youtput = float((yinput / abs(sizeY)) * resY);
    return resY-youtput



## Dataframe Handling

### Combining the datasets

In [2]:
# File names
meta_files = ["esea_meta_demos.part1.csv", "esea_meta_demos.part2.csv"]
grenade_files = ["esea_master_grenades_demos.part1.csv", "esea_master_grenades_demos.part2.csv"]

# Read and combine metadata files
meta_dfs = [pd.read_csv(os.path.join(ESEA_DEMO_PATH, file)) for file in meta_files]
metadata = pd.concat(meta_dfs, ignore_index=True)

# Read and combine grenade files
grenade_dfs = [pd.read_csv(os.path.join(ESEA_DEMO_PATH, file)) for file in grenade_files]
grenades = pd.concat(grenade_dfs, ignore_index=True)

# Drop unwanted columns from the grenades dataframe
grenades = grenades.drop(columns=["att_team", "vic_team", "att_id", "vic_id", "bomb_site", "hitbox"], errors="ignore")

# Merge grenades with metadata based on matching file and round columns
combined_df = pd.merge(
    grenades,
    metadata[["file", "round", "map", "round_type", "winner_side"]],
    on=["file", "round"],
    how="left"
)

# Save combined dataframe to a CSV for convenience (optional)
if WRITE_FILES:
    output_file = os.path.join(ESEA_DEMO_PATH, "combined_grenades_metadata.csv")
    combined_df.to_csv(output_file, index=False)
    print(f"New Data frame saved at :{WRITE_FILES}")

combined_df.head()



,file,round,seconds,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y,map,round_type,winner_side
0,esea_match_13770997.dem,1,153.1602,CounterTerrorist,NaN,0,0,True,Smoke,0,NaN,-1618.146,-66.00259,-949.8569,-340.3019,NaN,NaN,de_overpass,PISTOL_ROUND,Terrorist
1,esea_match_13770997.dem,2,184.7945,Terrorist,CounterTerrorist,70,0,False,HE,0,0.0,-1719.904,-2357.64700,-2774.6650,-1603.9430,-2741.25,-1523.163,de_overpass,ECO,Terrorist
2,esea_match_13770997.dem,2,186.8617,CounterTerrorist,NaN,0,0,False,HE,0,NaN,-1036.352,492.16760,-466.8676,-356.9641,NaN,NaN,de_overpass,ECO,Terrorist
3,esea_match_13770997.dem,2,187.1122,CounterTerrorist,NaN,0,0,False,HE,0,NaN,-855.077,438.69090,-459.0147,-543.8581,NaN,NaN,de_overpass,ECO,Terrorist
4,esea_match_13770997.dem,2,191.0587,Terrorist,NaN,0,0,False,Molotov,0,NaN,-2617.490,-1832.40700,-2743.5610,-927.2995,NaN,NaN,de_overpass,ECO,Terrorist


### Storing individual map's data

In [15]:
if WRITE_FILES:
    # Load combined dataframe
    combined_df_path = os.path.join(ESEA_DEMO_PATH, "combined_grenades_metadata.csv")
    combined_df = pd.read_csv(combined_df_path)

# Load map coordinate data
map_coord_data = pd.read_csv(MAP_COORD_DATA_PATH)

# Dictionary to store individual dataframes for each map
map_dataframes = {}

# Map coordinate transformation logic
for map_name in COMPETITIVE_MAPS:
    map_df = combined_df[combined_df["map"] == map_name]
    
    # Retrieve map-specific coordinate details
    map_details = map_coord_data[map_coord_data["Map"] == map_name]
    if map_details.empty:
        print(f"No coordinate data found for {map_name}. Skipping...")
        continue
    
    # Extract coordinate transformation parameters
    startX = map_details["StartX"].values[0]
    endX = map_details["EndX"].values[0]
    startY = map_details["StartY"].values[0]
    endY = map_details["EndY"].values[0]
    resX = map_details["ResX"].values[0]
    resY = map_details["ResY"].values[0]
    
    # Apply coordinate transformation
    map_df["att_pos_x"] = map_df["att_pos_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["att_pos_y"] = map_df["att_pos_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    map_df["nade_land_x"] = map_df["nade_land_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["nade_land_y"] = map_df["nade_land_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    map_df["vic_pos_x"] = map_df["vic_pos_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["vic_pos_y"] = map_df["vic_pos_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    
    # Store the transformed dataframe
    map_dataframes[map_name] = map_df
    
    if WRITE_FILES:
        # Optionally save the transformed map dataframe to a CSV
        output_file = os.path.join(ESEA_DEMO_PATH,'map_grenade_data',f"{map_name}_grenades_transformed.csv")
        map_df.to_csv(output_file, index=False)

# Example: Accessing the transformed dataframe for "de_mirage"
de_mirage_df = map_dataframes.get("de_mirage", pd.DataFrame())
de_mirage_df.head()

No coordinate data found for de_nuke. Skipping...


,file,round,seconds,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y,map,round_type,winner_side
1339,esea_match_13779770.dem,2,224.1890,CounterTerrorist,NaN,0,0,False,Smoke,0,NaN,489.432829,796.030363,695.206078,642.629200,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist
1340,esea_match_13779770.dem,2,225.7543,CounterTerrorist,NaN,0,0,False,HE,0,NaN,552.523867,755.225081,732.599288,667.528537,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist
1341,esea_match_13779770.dem,2,227.2570,CounterTerrorist,NaN,0,0,False,Incendiary,0,NaN,271.383559,352.924696,294.389319,189.557308,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist
1342,esea_match_13779770.dem,2,228.0554,CounterTerrorist,NaN,0,0,False,HE,0,NaN,502.607191,788.164722,728.375792,626.179324,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist
1343,esea_match_13779770.dem,2,236.2108,CounterTerrorist,NaN,0,0,False,Smoke,0,NaN,208.976259,183.114005,432.667851,196.930490,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist


In [17]:
# Checking Map Names

# unique_maps = combined_df["map"].unique()
# print("Unique map names in the dataset:")
# for map_name in unique_maps:
#     print(map_name)

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from ipywidgets import Tab, VBox, HBox, Output, Button
import os

# Dictionary to store map figures
map_figures = {}

# Function to plot decoy grenade positions on the map image for both sides
def plot_decoy_grenades(ax, data, map_image_path):
    
    filtered_data_t = data[(data["nade"] == "decoy") & (data["att_side"] == "Terrorist")]
    filtered_data_ct = data[(data["nade"] == "decoy") & (data["att_side"] == "CounterTerrorist")]


    # Load the map image
    img = imread(map_image_path)
    ax.imshow(img, extent=[0, 1024, 1024, 0], origin='upper')  # Adjust extent based on your image dimensions

    # Scatter plot the points
    ax.scatter(filtered_data_t["att_pos_x"], filtered_data_t["att_pos_y"],
               marker='o', color='orange', label='Terrorist', alpha=0.05, s=1)
    ax.scatter(filtered_data_ct["att_pos_x"], filtered_data_ct["att_pos_y"],
               marker='o', color='blue', label='CounterTerrorist', alpha=0.05, s=1)

    ax.set_title("Decoy Grenade Positions")
    ax.legend()
    ax.axis('off')  # Remove axes for better visualization

# Create tabs for each map
tab_children = []
map_names = list(map_dataframes.keys())

for map_name, map_df in map_dataframes.items():
    # Path to the map image
    map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
    if not os.path.exists(map_image_path):
        print(f"Map image for {map_name} not found. Skipping...")
        continue

    # Create a figure for the current map
    fig, ax = plt.subplots(figsize=(12, 6))
    plot_decoy_grenades(ax, map_df, map_image_path)

    # Store the figure for saving
    map_figures[map_name] = [fig]

    # Display the figure in an Output widget
    output = Output()
    with output:
        plt.show()

    tab_children.append(output)

# Create Tab widget
tabs = Tab(children=tab_children)
for idx, map_name in enumerate(map_names):
    tabs.set_title(idx, map_name)

# Function to save the current tab's visualization
def save_current_tab_images(button):
    selected_tab = tabs.selected_index
    selected_map = map_names[selected_tab]
    figs = map_figures.get(selected_map, [])
    
    for idx, fig in enumerate(figs):
        save_path = f"{selected_map}_decoy_grenades_{idx}.png"
        fig.savefig(save_path)
        print(f"Saved: {save_path}")

# Create a Save button
save_button = Button(description="Save Current Tab Images", button_style="success")
save_button.on_click(save_current_tab_images)

# Display the dashboard with the save button
VBox([tabs, save_button])



NameError: name 'filtered_data_t' is not defined

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from ipywidgets import Tab, VBox, HBox, Output, Button
import os

# Path to map images
MAP_IMAGES_PATH = "../csgo_mm_data/map_images"

# Dictionary to store map figures
map_figures = {}

# Function to plot attacker positions on the map image for a grenade type
def plot_att_pos_on_map(ax, data, grenade_type, side, color, map_image_path):
    filtered_data = data[(data["nade"] == grenade_type) & (data["att_side"] == side)]
    
    # Load the map image
    img = imread(map_image_path)
    ax.imshow(img, extent=[0, 1024, 1024, 0], origin='upper')  # Adjust extent based on your image dimensions
    
    # Scatter plot the points
    
    ax.scatter(filtered_data["att_pos_x"], filtered_data["att_pos_y"],marker='o', color=color, label=side, alpha=0.05,s=1)
    # ax.invert_yaxis()
    # ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_title(f"{grenade_type} - {side}")
    # ax.axis('off')
    

# Create tabs for each map
tab_children = []
map_names = list(map_dataframes.keys())

for map_name, map_df in map_dataframes.items():
    # Path to the map image
    map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
    if not os.path.exists(map_image_path):
        print(f"Map image for {map_name} not found. Skipping...")
        continue

    # Get unique grenade types
    grenade_types = map_df["nade"].unique()

    # Create rows for grenade types
    rows = []
    fig_list = []  # Store figures for saving
    for grenade_type in grenade_types:
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # Plot Terrorist side
        plot_att_pos_on_map(axes[0], map_df, grenade_type, "Terrorist", "orange", map_image_path)
        
        # Plot Counter-Terrorist side
        plot_att_pos_on_map(axes[1], map_df, grenade_type, "CounterTerrorist", "blue", map_image_path)
        
        # Add the figure to the row
        output = Output()
        with output:
            plt.show()
        rows.append(output)
        fig_list.append(fig)

    # Save figures for the current map
    map_figures[map_name] = fig_list

    # Arrange grenade rows in a VBox
    vbox = VBox(rows)
    tab_children.append(vbox)

# Create Tab widget
tabs = Tab(children=tab_children)
for idx, map_name in enumerate(map_names):
    tabs.set_title(idx, map_name)

# Function to save the current tab's visualizations
def save_current_tab_images(button):
    selected_tab = tabs.selected_index
    selected_map = map_names[selected_tab]
    figs = map_figures.get(selected_map, [])
    
    for idx, fig in enumerate(figs):
        save_path = f"{selected_map}_grenade_{idx}.png"
        fig.savefig(save_path)
        print(f"Saved: {save_path}")

# Create a Save button
save_button = Button(description="Save Current Tab Images", button_style="success",)
save_button.on_click(save_current_tab_images)

# Display the dashboard with the save button
VBox([tabs, save_button])